In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from keras import layers
from keras import models
from keras.utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the path to the dataset folder
dataset_path = "/content/drive/MyDrive/MMU-Iris-Database"
data_path = "/content/drive/MyDrive/AMF Iris Dataset"
casia_path = "/content/drive/MyDrive/casia-iris-syc"

In [ ]:
# Initialize lists to store images and labels
images = []
person_labels = []
test_data = []
test_person_label = []

img_dim = {'width': 225, 'height': 225}

i = 1
person_id = 0


In [ ]:
# Loop through each person's folder
for person_folder in os.listdir(dataset_path):
    if('.txt' in person_folder): continue

    for eye_side_folder in os.listdir(os.path.join(dataset_path, person_folder)):
        eye_side = 0 if eye_side_folder == "left" else 1 # Assign labels for left (0) and right (1)
        for image_filename in os.listdir(os.path.join(dataset_path, person_folder, eye_side_folder)):
            if('.db' in image_filename): continue
            image_path = os.path.join(dataset_path, person_folder, eye_side_folder, image_filename)

            # Load and preprocess the image
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB) # Convert to grayscale (if not already)
            img = cv2.resize(img, (img_dim['width'], img_dim['height'])) # Resize to a common size

            img = img / 255.0

            if i % 5 == 0:
                test_data.append(img)
                test_person_label.append(person_id)

            else:
                images.append(img)
                person_labels.append(person_id)

            i += 1
    person_id += 1


print('Total of Original images:', len(images))
print('Total of test data (20% original data):', len(test_data))
print('test_labesl', test_person_label)


Total of Original images: 360
Total of test data (20% original data): 90
test_labesl [0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44]


In [ ]:
# Loop through each person's folder
for person_folder in os.listdir(data_path):
    if('.txt' in person_folder): continue

    for image_filename in os.listdir(os.path.join(data_path, person_folder)):
            if('.db' in image_filename): continue
            image_path = os.path.join(data_path, person_folder, image_filename)

            # Load and preprocess the image
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB) # Convert to grayscale (if not already)
            img = cv2.resize(img, (img_dim['width'], img_dim['height'])) # Resize to a common size

            img = img / 255.0

            if i % 5 == 0:
                test_data.append(img)
                test_person_label.append(person_id)

            else:
                images.append(img)
                person_labels.append(person_id)

            i += 1
    person_id += 1


print('Total of Original images:', len(images))
print('Total of test data (20% original data):', len(test_data))
print('test_labesl', test_person_label)
print(len(test_person_label)+len(person_labels))

Total of Original images: 792
Total of test data (20% original data): 198
test_labesl [0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47, 47, 48, 48, 49, 49, 50, 50, 51, 51, 52, 52, 53, 53, 54, 54, 55, 55, 56, 56, 57, 57, 58, 58, 59, 59, 60, 60, 61, 61, 62, 62, 63, 63, 64, 64, 65, 65, 66, 66, 67, 67, 68, 68, 69, 69, 70, 70, 71, 71, 72, 72, 73, 73, 74, 74, 75, 75, 76, 76, 77, 77, 78, 78, 79, 79, 80, 80, 81, 81, 82, 82, 83, 83, 84, 84, 85, 85, 86, 86, 87, 87, 88, 88, 89, 89, 90, 90, 91, 91, 92, 92, 93, 93, 94, 94, 95, 95, 96, 96, 97, 97, 98, 98]
990


In [ ]:
images = np.array(images ,  dtype= 'float32')
person_labels = np.array(person_labels)
test_data = np.array(test_data , dtype= 'float32')
test_person_label = np.array(test_person_label)

In [ ]:
#build cnn
model = tf.keras.models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_dim['width'], img_dim['height'], 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(99, activation='softmax'))


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

test_data = np.array(test_data)
test_person_label = np.array(test_person_label)

history = model.fit(images, person_labels, epochs=77  ,batch_size=32, validation_data=(test_data, test_person_label))

test_loss, test_acc = model.evaluate(test_data,  test_person_label, verbose=2)
print(test_acc)

Epoch 1/77


C:\Users\windos 10\anaconda3\Lib\site-packages\keras\src\backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


25/25 [==============================] - 82s 3s/step - loss: 4.6367 - accuracy: 0.0215 - val_loss: 4.2788 - val_accuracy: 0.1111
Epoch 2/77
25/25 [==============================] - 79s 3s/step - loss: 3.5991 - accuracy: 0.1894 - val_loss: 2.2289 - val_accuracy: 0.4848
Epoch 3/77
25/25 [==============================] - 80s 3s/step - loss: 1.5573 - accuracy: 0.6162 - val_loss: 1.4003 - val_accuracy: 0.7273
Epoch 4/77
25/25 [==============================] - 84s 3s/step - loss: 0.6264 - accuracy: 0.8434 - val_loss: 1.2720 - val_accuracy: 0.7626
Epoch 5/77
25/25 [==============================] - 80s 3s/step - loss: 0.2288 - accuracy: 0.9343 - val_loss: 1.0118 - val_accuracy: 0.8081
Epoch 6/77
25/25 [==============================] - 78s 3s/step - loss: 0.1457 - accuracy: 0.9545 - val_loss: 1.0688 - val_accuracy: 0.7727
Epoch 7/77
25/25 [==============================] - 79s 3s/step - loss: 0.1231 - accuracy: 0.9760 - val_loss: 1.3964 - val_accuracy: 0.7677
Epoch 8/77
25/25 [=============

Epoch 58/77
25/25 [==============================] - 111s 4s/step - loss: 5.2229e-07 - accuracy: 1.0000 - val_loss: 1.9313 - val_accuracy: 0.8434
Epoch 59/77
25/25 [==============================] - 103s 4s/step - loss: 4.6299e-07 - accuracy: 1.0000 - val_loss: 1.9315 - val_accuracy: 0.8434
Epoch 60/77
25/25 [==============================] - 106s 4s/step - loss: 4.1257e-07 - accuracy: 1.0000 - val_loss: 1.9365 - val_accuracy: 0.8434
Epoch 61/77
25/25 [==============================] - 20212s 842s/step - loss: 3.7012e-07 - accuracy: 1.0000 - val_loss: 1.9378 - val_accuracy: 0.8434
Epoch 62/77
25/25 [==============================] - 88s 4s/step - loss: 3.4167e-07 - accuracy: 1.0000 - val_loss: 1.9417 - val_accuracy: 0.8434
Epoch 63/77
25/25 [==============================] - 98s 4s/step - loss: 3.1067e-07 - accuracy: 1.0000 - val_loss: 1.9436 - val_accuracy: 0.8434
Epoch 64/77
25/25 [==============================] - 101s 4s/step - loss: 2.8824e-07 - accuracy: 1.0000 - val_loss: 1.9480

In [ ]:
print(test_acc)
#print(test_person_label)
predict = model.predict(test_data)
predict = np.argmax(predict, axis = -1)
#for i in test_person_label :
    #print(i , test_person_label[i] == predict[i])

0.8535353541374207
7/7 [==============================] - 4s 503ms/step
